In [ ]:
import ipywidgets as widgets  # type: ignore
from PIL import Image, ImageDraw
from io import BytesIO

import board
import board_data
import player
import scotland_yard


# This notebook provides a GUI for the game, implemented
# using the `ipywidgets` library.

In [ ]:
# Utility functions to render the board image

LARGE_RING = 16
SMALL_RING = 12

def draw_rings(image_draw: ImageDraw.ImageDraw, x: int, y: int, color: str) -> None:
    """
    Draw two small rings, to represent the position of a player.
    `x` and `y` are the coordinates of the center of the rings, and `color`
    the color of the "internal" ring - the "external" one being white.
    """
    x0 = x - LARGE_RING
    y0 = y - LARGE_RING
    x1 = x + LARGE_RING
    y1 = y + LARGE_RING
    image_draw.ellipse([(x0, y0), (x1, y1)], outline="white", width=8)
    x0 = x - SMALL_RING
    y0 = y - SMALL_RING
    x1 = x + SMALL_RING
    y1 = y + SMALL_RING
    image_draw.ellipse([(x0, y0), (x1, y1)], outline=color, width=4)

# Dictonary from station number to (x, y) position
STATION_POSITIONS: dict[board.StationNumber, tuple[int, int]] = {
    station_num: (x, y) for station_num, x, y in board_data.POSITIONS
}


def compute_board_image(game: scotland_yard.Game) -> bytes:
    """
    Creates an image of the board, starting from the base png file
    and adding rings to represent the positions of the players.
    The image is returned as bytes, using the png format.
    """
    board_image = Image.open("board.png")
    board_draw = ImageDraw.Draw(board_image)
    for detective in game.detectives:
        station = detective.position
        x, y = STATION_POSITIONS[station]
        draw_rings(board_draw, x, y, detective.color.value)
    x, y = STATION_POSITIONS[game.mister_x.position]
    draw_rings(board_draw, x, y, game.mister_x.color.value)
    bytes = BytesIO()
    board_image.save(bytes, format="PNG")
    return bytes.getvalue()

In [ ]:
# Game creation

game = scotland_yard.Game(
    detectives=[
        (color, f"{color.value} detective")
        for color in player.Color
        if color != player.Color.BLACK
    ],
    mister_x_name="Mr X",
)

In [ ]:
# Graphical user interface

UNKNOWN = "???"

TicketsMap = dict[player.Color, dict[board.StationKind, widgets.Label]]


def create_panel(game: scotland_yard.Game) -> tuple[widgets.Widget, TicketsMap]:
    """
    Creates the panel to be displayed on the right-hand side of the graphical
    interface. The panel contains the information about tickets for all players.
    Returns the widget for the all panel, as well as a dictionary from player
    colors to the widget displaying the ticket numbers.
    """
    ticket_map: dict[player.Color, dict[board.StationKind, widgets.Label]] = {}
    grid = widgets.GridspecLayout(n_rows=(len(game.detectives) + 2), n_columns=4)
    grid[0, 0] = widgets.Label(value="")
    grid[0, 1] = widgets.Label(value="bus")
    grid[0, 2] = widgets.Label(value="taxi")
    grid[0, 3] = widgets.Label(value="underground")
    for row, player_ in enumerate(game.detectives + [game.mister_x], start=1):
        ticket_map[player_.color] = {}
        grid[row, 0] = widgets.Label(
            value=f"{player_.name}", layout=widgets.Layout(width="128px")
        )
        for column, kind in enumerate(board.StationKind, start=1):
            ticket_widget = widgets.Text(
                value=UNKNOWN, disabled=True, layout=widgets.Layout(width="64px")
            )
            grid[row, column] = ticket_widget
            ticket_map[player_.color][kind] = ticket_widget
    return grid, ticket_map


# components of the interface
panel, held_tickets_widgets = create_panel(game)
log = widgets.Select(options=[], disabled=True, layout=widgets.Layout(width="368px"))
play_button = widgets.Button(description="Play")
black_tickets = widgets.Text(
    value=UNKNOWN,
    disabled=True,
    layout=widgets.Layout(width="64px"),
)
double_tickets = widgets.Text(
    value=UNKNOWN,
    disabled=True,
    layout=widgets.Layout(width="64px"),
)
message = widgets.Text(value=UNKNOWN, disabled=True)
round = widgets.Text(value=UNKNOWN, disabled=True, layout=widgets.Layout(width="64px"))
moves = widgets.Dropdown(value="move-0", options=[f"move-{i}" for i in range(16)])

# main part of the interface
widgets.VBox(
    [
        widgets.HBox(
            [
                widgets.Image(value=compute_board_image(game), width=1008, height=756),
                widgets.VBox(
                    [
                        panel,
                        widgets.HBox(
                            [
                                widgets.Label(
                                    value="", layout=widgets.Layout(width="32px")
                                ),
                                widgets.Label(
                                    value="black tickets",
                                    layout=widgets.Layout(width="128px"),
                                ),
                                black_tickets,
                            ]
                        ),
                        widgets.HBox(
                            [
                                widgets.Label(
                                    value="", layout=widgets.Layout(width="32px")
                                ),
                                widgets.Label(
                                    value="double tickets",
                                    layout=widgets.Layout(width="128px"),
                                ),
                                double_tickets,
                            ]
                        ),
                        widgets.Label(value=""),
                        widgets.Label(value="Log:"),
                        log,
                    ]
                ),
            ]
        ),
        widgets.HBox(
            [
                message,
                widgets.Label(value="Round:"),
                round,
                widgets.Label(value="Move:"),
                moves,
                play_button,
            ]
        ),
    ]
)